In [1]:
from numpy import *

In [2]:
def loadDataSet(fileName):    # 通用函数，用来解析以 tab 键分隔的 floats（浮点数）
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float,curLine)    # 映射所有的元素为 float（浮点数）类型
        dataMat.append(fltLine)
    return dataMat

In [3]:
# 计算两个向量的欧式距离（可根据场景选择）
def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) # la.norm(vecA-vecB)

In [16]:
# 为给定数据集构建一个包含 k 个随机质心的集合。随机质心必须要在整个数据集的边界之内，这可以通过找到数据集每一维的最小和最大值来完成。然后生成 0~1.0 之间的随机数并通过取值范围和最小值，以便确保随机点在数据的边界之内。
def randCent(dataSet, k):
    n = shape(dataSet)[1] # 列的数量 shape[0]第一维长度 shape[1]第二维长度
    centroids = mat(zeros((k,n))) # 创建k个质心矩阵
    for j in range(n): # 创建随机簇质心，并且在每一维的边界内
        minJ = min(dataSet[:,j])    # 最小值
        rangeJ = float(max(dataSet[:,j]) - minJ)    # 范围 = 最大值 - 最小值
        centroids[:,j] = mat(minJ + rangeJ * random.rand(k,1))    # 随机生成
    return centroids

In [6]:
# k-means 聚类算法
# 该算法会创建k个质心，然后将每个点分配到最近的质心，再重新计算质心。
# 这个过程重复数次，知道数据点的簇分配结果不再改变位置。
# 运行结果（多次运行结果可能会不一样，可以试试，原因为随机质心的影响，但总的结果是对的， 因为数据足够相似，也可能会陷入局部最小值）
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]    # 行数
    clusterAssment = mat(zeros((m, 2)))    # 创建一个与 dataSet 行数一样，但是有两列的矩阵，用来保存簇分配结果
    centroids = createCent(dataSet, k)    # 创建质心，随机k个质心
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):    # 循环每一个数据点并分配到最近的质心中去
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])    # 计算数据点到质心的距离
                if distJI < minDist:    # 如果距离比 minDist（最小距离）还小，更新 minDist（最小距离）和最小质心的 index（索引）
                    minDist = distJI; minIndex = j
            if clusterAssment[i, 0] != minIndex:    # 簇分配结果改变
                clusterChanged = True    # 簇改变
                clusterAssment[i, :] = minIndex,minDist**2    # 更新簇分配结果为最小质心的 index（索引），minDist（最小距离）的平方
        print centroids
        for cent in range(k): # 更新质心
            ptsInClust = dataSet[nonzero(clusterAssment[:, 0].A==cent)[0]] # 获取该簇中的所有点
            centroids[cent,:] = mean(ptsInClust, axis=0) # 将质心修改为簇中所有点的平均值，mean 就是求平均值的
    return centroids, clusterAssment

# 二分 KMeans 聚类算法, 基于 kMeans 基础之上的优化，以避免陷入局部最小值
def biKMeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2))) # 保存每个数据点的簇分配结果和平方误差
    centroid0 = mean(dataSet, axis=0).tolist()[0] # 质心初始化为所有数据点的均值
    centList =[centroid0] # 初始化只有 1 个质心的 list
    for j in range(m): # 计算所有数据点到初始质心的距离平方误差
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k): # 当质心数量小于 k 时
        lowestSSE = inf
        for i in range(len(centList)): # 对每一个质心
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:] # 获取当前簇 i 下的所有数据点
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas) # 将当前簇 i 进行二分 kMeans 处理
            sseSplit = sum(splitClustAss[:,1]) # 将二分 kMeans 结果中的平方和的距离进行求和
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1]) # 将未参与二分 kMeans 分配结果中的平方和的距离进行求和
            print "sseSplit, and notSplit: ",sseSplit,sseNotSplit
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        # 找出最好的簇分配结果    
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) # 调用二分 kMeans 的结果，默认簇是 0,1. 当然也可以改成其它的数字
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit # 更新为最佳质心
        print 'the bestCentToSplit is: ',bestCentToSplit
        print 'the len of bestClustAss is: ', len(bestClustAss)
        # 更新质心列表
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0] # 更新原质心 list 中的第 i 个质心为使用二分 kMeans 后 bestNewCents 的第一个质心
        centList.append(bestNewCents[1,:].tolist()[0]) # 添加 bestNewCents 的第二个质心
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss # 重新分配最好簇下的数据（质心）以及SSE
    return mat(centList), clusterAssment

In [7]:
def testBasicFunc():
    # 加载测试数据集
    datMat = mat(loadDataSet('10.KMeans/testSet.txt'))

    # 测试 randCent() 函数是否正常运行。
    # 首先，先看一下矩阵中的最大值与最小值
    print 'min(datMat[:, 0])=', min(datMat[:, 0])
    print 'min(datMat[:, 1])=', min(datMat[:, 1])
    print 'max(datMat[:, 1])=', max(datMat[:, 1])
    print 'max(datMat[:, 0])=', max(datMat[:, 0])

    # 然后看看 randCent() 函数能否生成 min 到 max 之间的值
    print 'randCent(datMat, 2)=', randCent(datMat, 2)

    # 最后测试一下距离计算方法
    print ' distEclud(datMat[0], datMat[1])=', distEclud(datMat[0], datMat[1])

def testKMeans():
    # 加载测试数据集
    datMat = mat(loadDataSet('10.KMeans/testSet.txt'))

    # 该算法会创建k个质心，然后将每个点分配到最近的质心，再重新计算质心。
    # 这个过程重复数次，知道数据点的簇分配结果不再改变位置。
    # 运行结果（多次运行结果可能会不一样，可以试试，原因为随机质心的影响，但总的结果是对的， 因为数据足够相似）
    myCentroids, clustAssing = kMeans(datMat, 4)

    print 'centroids=', myCentroids

def testBiKMeans():
    # 加载测试数据集
    datMat = mat(loadDataSet('10.KMeans/testSet2.txt'))

    centList, myNewAssments = biKMeans(datMat, 3)

    print 'centList=', centList

In [14]:
    # 测试基础的函数
testBasicFunc()
print "=================================================="
    # 测试 kMeans 函数
testKMeans()
print "=================================================="
    # 测试二分 biKMeans 函数
testBiKMeans()


min(datMat[:, 0])= [[-5.379713]]
min(datMat[:, 1])= [[-4.232586]]
max(datMat[:, 1])= [[ 5.1904]]
max(datMat[:, 0])= [[ 4.838138]]
randCent(datMat, 2)= nnnnnn
2
[[ 0.27656738 -3.1093461 ]
 [ 0.27512854  4.11072286]]
 distEclud(datMat[0], datMat[1])= 5.18463281668
[[-1.33886155  0.98584083]
 [ 4.49117437  2.82224818]
 [-0.01959672  2.6091127 ]
 [-1.1216449  -3.96455993]]
[[-2.6509714   2.39277907]
 [ 3.35821883  1.83169994]
 [ 0.1399645   3.52317367]
 [-0.8757714  -3.05132209]]
[[-2.70030241  2.70457282]
 [ 3.28916832  1.87925674]
 [ 0.63371075  3.74202925]
 [-0.83188333 -2.97988206]]
[[-2.54951105  2.75812458]
 [ 3.34647022  1.79546189]
 [ 1.40899843  3.84245271]
 [-0.83188333 -2.97988206]]
[[-2.46154315  2.78737555]
 [ 3.67314583  0.937859  ]
 [ 2.23432133  3.71816925]
 [-0.83188333 -2.97988206]]
[[-2.46154315  2.78737555]
 [ 3.6509195  -0.5281174 ]
 [ 2.5212765   3.49464725]
 [-1.05611215 -3.00107638]]
[[-2.46154315  2.78737555]
 [ 3.1666855  -2.38897356]
 [ 2.54391447  3.21299611]
 [